# YCbCr - Y: Luminance; Cb: Chrominance-Blue; and Cr: Chrominance-Red Colorspace
Stough, 202-

- [skimage's color module](https://scikit-image.org/docs/dev/api/skimage.color.html)
- [What is YCbCr?](https://en.wikipedia.org/wiki/YCbCr)
- [Why YCbCr?](https://makarandtapaswi.wordpress.com/2009/07/20/why-the-rgb-to-ycbcr/)

The YCBCr colorspace is also oriented around adjusting color according to human vision
*perception* as opposed to mechanism (r-g-b stimulus).

<img src="https://i.pinimg.com/originals/bf/2d/0e/bf2d0e090f386111cae1d8e06d4f0015.jpg" style="width:auto;height:200px"/> <img src="https://d3i71xaburhd42.cloudfront.net/03e0ea040482660aa7f4ef73b893d339b2700bbe/3-Figure2-1.png" style="width:auto;height:200px"/> 
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/YCbCr-CbCr_Scaled_Y50.png/600px-YCbCr-CbCr_Scaled_Y50.png" style="width:auto;height:200px"/> 


In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import skimage.color as color
import matplotlib.colors as mcolors

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils
from vis_utils import (vis_rgb_cube,
                       vis_ybr_cube,
                       vis_hists,
                       lab_uniform)

In [ ]:
IMAGE = 'mountainSpring.jpg'
# IMAGE = 'peppers.png'

# Let's construct our own colormaps
cm_KR = mcolors.LinearSegmentedColormap.from_list('kr', [(0,0,0), (1, 0, 0)], 256)
cm_KG = mcolors.LinearSegmentedColormap.from_list('kg', [(0,0,0), (0, 1, 0)], 256)
cm_KB = mcolors.LinearSegmentedColormap.from_list('kb', [(0,0,0), (0, 0, 1)], 256)

In [ ]:
Irgb = plt.imread('../dip_pics/' + IMAGE)

# Image of uniform random distributed color
# Irgb = np.random.random((100,100,3))

&nbsp;

## Cb and Cr colormaps
I'm going to be careful here with the colormap to have a perceptually constant
transition between yello and blue (Cb) and cyan and red (Cr). I don't want one side
to be perceptually brighter than the other.

In [ ]:
# Let's construct our own colormaps for the a and b dimensions.
# I'm going to be careful with my color,  
colorsCb = [(.7, .7, 0), (.6,.6,1)]
colorsCr = [(0, .7, .7), (1,.4,.4)]

cmCb = mcolors.LinearSegmentedColormap.from_list('Cb', lab_uniform(colorsCb), 256)
cmCr = mcolors.LinearSegmentedColormap.from_list('Cr', lab_uniform(colorsCr), 256)

In [ ]:
f, ax = plt.subplots(3,3, figsize = (8,6), sharex=True, sharey=True)

Iybr = color.rgb2ycbcr(Irgb)

# Row 0: just the full color image
ax[0][1].imshow(Irgb)
ax[0][0].set_axis_off()
ax[0][2].set_axis_off()

# Row 1: r g b channels
for i in range(3):
    ax[1,i].imshow(Irgb[...,i], cmap=[cm_KR, cm_KG, cm_KB][i])
    ax[1,i].set_title(['Red', 'Green', 'Blue'][i])


# Now to display the HSV space.

ax[2,0].imshow(Iybr[...,0], cmap='gray')
ax[2,0].set_title('Y')

ax[2,1].imshow(Iybr[...,1], cmap=cmCb)
ax[2,1].set_title('Cb - yellow->blue')

ax[2,2].imshow(Iybr[...,2], cmap=cmCr)
ax[2,2].set_title('Cr - cyan->red')

plt.tight_layout()
f.canvas.set_window_title('RGB vs HSV')
plt.show()

&nbsp;

## Let's visualize in 3D

In [ ]:
vis_ybr_cube(Irgb)

In [ ]:
vis_ybr_cube(np.random.random((100,100,3)))

&nbsp;

## View the Cb-Cr plane for various Y

The visualization of the YCbCr space is a bit weird, because it looks just like the rgb color cube, but maybe
rotated, translated, and stretched a little. So I'm trying here to visualize the CbCr plane for any luma Y. 
[skimage](https://scikit-image.org/docs/dev/api/skimage.color.html#skimage.color.rgb2ycbcr) notes that Y 
must be in 16-235. Also, when you transform back you could get out-of-range values for r,g,b. Normally these
get clipped to show a CbCr plane like below, but a lot of that plane is invalid. So here, I'll show the whole 
(clipped to 0-1) plane and then also just the valid part (within the color cube). 

<img src="https://d3i71xaburhd42.cloudfront.net/03e0ea040482660aa7f4ef73b893d339b2700bbe/3-Figure2-1.png" style="width:auto;height:200px"/>  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/YCbCr-CbCr_Scaled_Y50.png/600px-YCbCr-CbCr_Scaled_Y50.png" style="width:auto;height:200px"/> 

In [ ]:
def clipRGB(rgb):
    toosmall = np.any(rgb<0, axis=2)
    toobig = np.any(rgb>1, axis=2)
    rgb[toobig | toosmall, :] = 0
    return rgb

In [ ]:
from ipywidgets import VBox, IntSlider

plt.ioff()
# plt.clf()

y_slider = IntSlider(
    orientation='horizontal',
    value=150,
    min=16,
    max=235,
    step=1,
    description='Y (Lightness)'
)


# Initialize globals
Cb, Cr = np.meshgrid(range(256), range(256), indexing='xy')
ybr = np.stack([150*np.ones((256,256)), Cb[::-1,:], Cr[::-1,:]], axis=2)
rgb = color.ycbcr2rgb(ybr)

# Make figure.
fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,2, figsize=(8,4), **fig_args) 
plt.tight_layout()


# display artists
ldisp = ax[0].imshow(np.clip(rgb,0,1))
ax[0].set_title('CbCr plane')
rdisp = ax[1].imshow(clipRGB(rgb))
rtext = ax[1].set_title(f'Valid at Y {150:03d}')

# Update function.
def update_image(change):
    global ybr, rgb, ldisp, rdisp, rtext
    
    ybr = np.stack([change.new*np.ones((256,256)), Cb[::-1,:], Cr[::-1,:]], axis=2)
    rgb = color.ycbcr2rgb(ybr)
    
    ldisp.set_array(np.clip(rgb,0,1))
    rdisp.set_array(clipRGB(rgb))
    rtext.set_text(f'Valid at Y {change.new:03d}')
    fig.canvas.draw()
    fig.canvas.flush_events()

y_slider.observe(update_image, names='value')

VBox([y_slider, fig.canvas])